# Implementación Práctica de Herramientas

## Objetivos de Aprendizaje
- Implementar herramientas que llamen a APIs externas o bases de datos
- Entender la diferencia entre herramientas simples y herramientas que devuelven datos estructurados
- Usar la anotación `Annotated` con `Field` para documentar parámetros
- Registrar y ejecutar herramientas con agentes ChatAgent
- Procesar respuestas de herramientas y analizarlas en el flujo del agente

## Conceptos Clave

### Herramientas Prácticas: Más Allá del Cálculo
Las herramientas reales no solo hacen cálculos; **conectan agentes con datos y servicios externos**.

**Patrón Esencial:**
```
Función → Datos Externos (API/BD) → Datos Estructurados → Agente → Análisis
```

### Diferencia Fundamental
| Aspecto | Herramientas Simples | Herramientas Prácticas |
|--------|---------------------|----------------------|
| **Entrada** | Parámetros simples | Parámetros que identifican recursos |
| **Proceso** | Cálculos o transformaciones | Llamadas a APIs, consultas BD |
| **Salida** | Valores calculados | Datos estructurados (JSON como string) |
| **Uso** | Matemáticas, lógica | Información, contexto, decisiones |
| **Ejemplo** | `calcular_warp(factor)` | `obtener_datos_sistema(nombre)` |

### La Importancia del Tipo `str` como Retorno
Las herramientas deben devolver **strings** porque:
- Los LLMs procesan texto, no objetos Python
- El formato JSON como string preserva la estructura
- El agente puede interpretar y analizar los datos

In [ ]:
# Importaciones necesarias
import os
import asyncio
from typing import Annotated
from pydantic import Field
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")

# Base de datos simulada de sistemas estelares de la Flota Estelar
BASE_DATOS_SISTEMAS = {
    "Vulcano": {
        "clase": "M",
        "poblacion": 6000000,
        "ambiente": "Desértico y lógico",
        "especie_dominante": "Vulcano"
    },
    "Terra": {
        "clase": "M",
        "poblacion": 8000000000,
        "ambiente": "Variado y dinámico",
        "especie_dominante": "Humano"
    },
    "Qo'noS": {
        "clase": "M",
        "poblacion": 4000000,
        "ambiente": "Montañoso y guerrero",
        "especie_dominante": "Klingon"
    },
    "Betazed": {
        "clase": "M",
        "poblacion": 3000000,
        "ambiente": "Tropical y empático",
        "especie_dominante": "Betazoide"
    },
    "Andoria": {
        "clase": "M",
        "poblacion": 2000000,
        "ambiente": "Glacial y diplomático",
        "especie_dominante": "Androide"
    }
}

print("✅ Configuración lista. Base de datos de 5 sistemas estelares cargada.")

✅ Configuración lista. Base de datos de 5 sistemas estelares cargada.


## Paso 1: Definir la Herramienta

La función `obtener_datos_sistema()` simula una llamada a la base de datos de la Flota Estelar.

**Puntos clave:**
- `Annotated[str, Field(...)]` documenta qué parámetro es necesario
- `-> str` indica que devolvemos datos como texto
- La función **simula** una llamada a API (en producción sería una llamada HTTP real)

In [2]:
# Definir la herramienta: obtener_datos_sistema
def obtener_datos_sistema(
    nombre_sistema: Annotated[str, Field(description="El nombre del sistema estelar a consultar")]
) -> str:
    """Obtener información actualizada sobre un sistema estelar de la Flota Estelar."""
    print(f"  📡 Consultando base de datos... sistema: {nombre_sistema}")
    
    # Verificar si el sistema existe en la base de datos
    if nombre_sistema not in BASE_DATOS_SISTEMAS:
        return f"⚠️ Error: Sistema '{nombre_sistema}' no encontrado en los registros de la Flota Estelar."
    
    # Simular consulta a API de la Flota Estelar
    # En producción: llamada HTTP a servidor remoto
    datos = BASE_DATOS_SISTEMAS[nombre_sistema]
    
    # Devolver como string formateado (importante para que el LLM pueda interpretarlo)
    resultado = f"""Sistema {nombre_sistema}:
    - Clase: {datos['clase']}
    - Población: {datos['poblacion']:,} habitantes
    - Ambiente: {datos['ambiente']}
    - Especie Dominante: {datos['especie_dominante']}"""
    
    return resultado

# Probar la función manualmente primero
print("\n📚 Test manual de la herramienta:")
print(obtener_datos_sistema("Vulcano"))
print("\n📚 Test con sistema no existente:")
print(obtener_datos_sistema("Romulo"))


📚 Test manual de la herramienta:
  📡 Consultando base de datos... sistema: Vulcano
Sistema Vulcano:
    - Clase: M
    - Población: 6,000,000 habitantes
    - Ambiente: Desértico y lógico
    - Especie Dominante: Vulcano

📚 Test con sistema no existente:
  📡 Consultando base de datos... sistema: Romulo
⚠️ Error: Sistema 'Romulo' no encontrado en los registros de la Flota Estelar.


## Paso 2: Crear el Agente con la Herramienta Registrada

El agente **automáticamente** invocará `obtener_datos_sistema` cuando sea necesario.

**Parámetros importantes:**
- `name`: Identificador único del agente
- `instructions`: Indica al agente cuándo y cómo usar la herramienta
- `tools`: Lista de funciones disponibles (pueden ser varias)

In [3]:
# Crear el cliente OpenAI
cliente = OpenAIChatClient(
    base_url=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    model_id=AZURE_OPENAI_DEPLOYMENT
)

# Crear el agente de exploración estelar
agente = ChatAgent(
    chat_client=cliente,
    name="ComputadoraExploracionEstelar",
    instructions="""Eres un oficial de comunicaciones de la Flota Estelar Unida.
Tu responsabilidad es proporcionar información precisa sobre sistemas estelares.
⚠️  IMPORTANTE: Responde de forma CONCISA (2-3 líneas máximo).
Cuando se te pida información sobre un sistema, SIEMPRE usa la herramienta obtener_datos_sistema.
Formatea la información claramente pero sin florituras.""",
    tools=[obtener_datos_sistema]  # Registrar la herramienta
)

print("✅ Agente 'ComputadoraExploracionEstelar' creado y listo para consultas.")

✅ Agente 'ComputadoraExploracionEstelar' creado y listo para consultas.


## Paso 3: Ejecutar Consultas con la Herramienta

El flujo de ejecución es:
1. **Usuario pregunta** sobre un sistema estelar
2. **Agente analiza** si necesita la herramienta
3. **Herramienta se invoca** automáticamente
4. **Datos se devuelven** al agente
5. **Agente procesa** y formatea la respuesta

In [10]:
# Función asincrónica para ejecutar las pruebas
async def ejecutar_pruebas():
    print("\n" + "="*70)
    print("🚀 EXPERIMENTO: Pruebas de Herramientas Prácticas de la Flota Estelar")
    print("="*70)
    
    # Tres consultas de diferentes perspectivas de la Flota
    consultas = [
        ("Picard", "¿Cuál es la información del sistema Vulcano?"),
        ("Geordi", "Dime sobre Qo'noS y su población."),
        ("Data", "Información del sistema Terra para mi análisis.")
    ]
    
    for oficial, consulta in consultas:
        print(f"\n📋 Consulta de {oficial}:")
        print(f"   {consulta}")
        print("-" * 70)
        
        # Ejecutar el agente
        resultado = await agente.run(consulta)
        
        # 🎯 ANÁLISIS DEL FLUJO
        print(f"\n   ✨ RESULTADO")
        
        print(resultado)
        
# Ejecutar
await ejecutar_pruebas()


🚀 EXPERIMENTO: Pruebas de Herramientas Prácticas de la Flota Estelar

📋 Consulta de Picard:
   ¿Cuál es la información del sistema Vulcano?
----------------------------------------------------------------------
  📡 Consultando base de datos... sistema: Vulcano

   ✨ RESULTADO
Sistema Vulcano: Clase M, ambiente desértico, población aproximada de 6 millones, especie dominante: vulcanos.

📋 Consulta de Geordi:
   Dime sobre Qo'noS y su población.
----------------------------------------------------------------------
  📡 Consultando base de datos... sistema: Qo'noS

   ✨ RESULTADO
Qo'noS es un planeta clase M con una población de 4 millones de habitantes, principalmente Klingon. Es montañoso y tiene una cultura guerrera.

📋 Consulta de Data:
   Información del sistema Terra para mi análisis.
----------------------------------------------------------------------
  📡 Consultando base de datos... sistema: Terra

   ✨ RESULTADO
Sistema Terra:
- Clase: M
- Población: 8,000,000,000
- Especie do

## Paso 4: Diagrama del Flujo de Ejecución

```
┌─────────────────────────────────────────────────────────────────┐
│ FLUJO DE EJECUCIÓN DE HERRAMIENTAS PRÁCTICAS                    │
└─────────────────────────────────────────────────────────────────┘

1️⃣  ENTRADA
    ↓
    Consulta del usuario: "¿Información sobre Vulcano?"
    
2️⃣  ANÁLISIS DEL AGENTE
    ↓
    "El usuario pregunta por un sistema estelar → necesito obtener_datos_sistema"
    
3️⃣  INVOCACIÓN DE HERRAMIENTA
    ↓
    obtener_datos_sistema(nombre_sistema="Vulcano")
    
4️⃣  CONSULTA A DATOS EXTERNOS
    ↓
    BASE_DATOS_SISTEMAS["Vulcano"] → {"clase": "M", "poblacion": 6M, ...}
    
5️⃣  FORMATEO Y RETORNO
    ↓
    return "Sistema Vulcano:\n  - Clase: M\n  - Población: 6,000,000..." (string)
    
6️⃣  PROCESAMIENTO POR AGENTE
    ↓
    Agente recibe datos y analiza: "Estos son datos de Vulcano"
    
7️⃣  GENERACIÓN DE RESPUESTA
    ↓
    Agente formula respuesta natural: "Vulcano es un planeta de Clase M..."
    
8️⃣  SALIDA
    ↓
    Respuesta final al usuario
```

## Resumen: Patrón de Herramientas Prácticas

### Características Principales

| Elemento | Descripción | Código |
|----------|-------------|--------|
| **Firma** | Función con tipos anotados | `def obtener_datos_sistema(nombre_sistema: Annotated[str, Field(...)])` |
| **Retorno** | Siempre `str` (datos formateados) | `-> str:` |
| **Datos Externos** | Consultamos bases de datos o APIs | `BASE_DATOS_SISTEMAS.get(nombre_sistema)` |
| **Formato** | Devolvemos strings legibles para LLM | `f"Sistema {nombre}: Clase {datos['clase']}..."` |
| **Registro** | Pasamos función al agente | `tools=[obtener_datos_sistema]` |
| **Invocación** | Automática cuando es necesaria | Agente detecta y llama sin intervención |

### Aprendizajes Clave

1. **Herramientas reales conectan agentes con datos y servicios**
2. **El tipo `str` como retorno es crucial** para que los LLMs interpreten datos
3. **Las anotaciones `Annotated` + `Field` documentan automáticamente parámetros**
4. **El framework maneja invocaciones automáticas** - no necesitas código especial
5. **El análisis de `tool_calls` permite auditar qué herramientas se usaron**

### Casos de Uso Reales
- Llamadas a APIs REST (clima, noticias, datos públicos)
- Consultas a bases de datos (usuarios, inventario, historiales)
- Búsquedas en índices (Elasticsearch, FAISS, vectores)
- Cálculos complejos con datos contextuales
- Integración con servicios externos (correo, SMS, webhooks)